In [18]:
from utils import read_csv_to_df, canonicalize_list, get_descriptors, normalize_df
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.inspection import permutation_importance

In [3]:
data_df = read_csv_to_df("ppb/ppb_filtered.csv")

raw_smiles = data_df["Fixed_SMILES"]
dropped_idx, mol_list = canonicalize_list(raw_smiles)
raw_descriptors = get_descriptors(mol_list)

raw_values = data_df["value"].drop(dropped_idx)

standardized_descriptors, scaler_x = normalize_df(raw_descriptors)
standardized_values, scaler_y = normalize_df(raw_values)

In [4]:
X = standardized_descriptors
y = standardized_values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Initialize individual regressors
model1 = LinearRegression()
model2 = RandomForestRegressor(random_state=42)
model3 = SVR(kernel="rbf")
model4 = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000, random_state=42)

# Create the ensemble model using VotingRegressor
ensemble_model = VotingRegressor(estimators=[
    # ('lr', model1),
    ('rf', model2),
    ('svr', model3),
    ('mlp', model4)  # Adding the MLPRegressor
])

# Fit the ensemble model to your training data
ensemble_model.fit(X_train, y_train)


VotingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                            ('svr', SVR()),
                            ('mlp',
                             MLPRegressor(hidden_layer_sizes=(100, 50),
                                          max_iter=1000, random_state=42))])

In [6]:
# Make predictions using the ensemble model
y_pred = ensemble_model.predict(X_test)

# Unscale y_pred (assuming y_pred is a NumPy array or you convert it)
y_pred_unscaled = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()

# Unscale y_test (convert y_test to a NumPy array first)
y_test_unscaled = scaler_y.inverse_transform(y_test.values.reshape(-1, 1)).flatten()

# Evaluate the model
mse = mean_squared_error(y_test_unscaled, y_pred_unscaled)
r2 = r2_score(y_test_unscaled, y_pred_unscaled)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 202.85845083156264
R-squared: 0.542575094916895


In [7]:
# Calculate permutation importance
perm_importance = permutation_importance(ensemble_model, X_test, y_test, n_repeats=10, random_state=42)

# Create a DataFrame to view feature importance
perm_importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

# Display the permutation importance
print("Permutation importance for the ensemble model:")
print(perm_importance_df)


Permutation importance for the ensemble model:
                            Feature  Importance
1                              LogP    0.567813
0                  Molecular Weight    0.183950
7                  Heavy Atom Count    0.129382
9                     Fraction Csp3    0.126513
10            Number of Heteroatoms    0.106795
4                     H-Bond Donors    0.087434
8               Aromatic Ring Count    0.083722
2                              TPSA    0.078972
13        Number of Saturated Rings    0.072425
15           Molecular Refractivity    0.068615
12      Number of Aliphatic Carbons    0.066175
3                   Number of Rings    0.063093
6                   Rotatable Bonds    0.047469
11        Number of Aliphatic Rings    0.045289
5                  H-Bond Acceptors    0.043947
14  Number of Aromatic Heterocycles    0.028011


Holdout


In [ ]:
data_df = read_csv_to_df("ppb/PPB_Human_testset.csv")
raw_smiles = data_df["SMILES"]
dropped_idx, mol_list = canonicalize_list(raw_smiles)

raw_descriptors = get_descriptors(mol_list)
standardized_descriptors, scaler_x = normalize_df(raw_descriptors)

In [17]:
holdout_pred = ensemble_model.predict(standardized_descriptors)
holdout_pred_unscaled = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
holdout_pred_unscaled

array([ 96.63961977,  94.06946155,  59.67656541,  85.50921279,
        93.55167917,  63.63739859,  83.914963  ,  74.89660192,
        96.08922264,  98.17718398,  97.55883446,  93.39995775,
        89.46817083,  50.80917484,  70.36394984,  91.4215601 ,
        95.23517616,  93.2420917 ,  80.34482772,  97.14474535,
        80.69424467,  91.57061609,  81.0164814 , 102.29660844,
       100.46086613,  98.67071607,  82.67520724,  83.32133608,
        96.30138932,  98.13121071,  79.3445673 ,  94.24103089,
        90.42943516,  62.72882918,  75.13802461,  89.61433872,
        74.40194112,  93.23095635,  69.88401948,  84.18504618,
        60.99602107,  65.32873703,  95.93468392,  75.48985586,
        80.38908716,  88.69393386,  88.55735468,  89.04222244,
        97.80136485,  74.95592404,  84.442163  ,  92.50163136,
        94.62241446,  95.95597738,  95.89649946,  98.24194579,
        99.56323674,  85.90408133,  92.92676348,  78.0256694 ,
        39.46371648,  43.38347978,  57.67669805,  91.65